In [1]:
!pip install workadays

In [7]:
import math
import pandas as pd
import datetime as dt
from datetime import datetime
from workadays import workdays as wd
from datetime import date

In [3]:
def truncate(num, n):
    integer = int(num * (10**n))/(10**n)
    return float(integer)

In [4]:
hoje = dt.date(2023,8,3)

In [9]:
# PRECIFICANDO NTN-B

cupom = 0.06
vencimento = dt.date(2026,8,15)
i = 5.0874
vna_ipca = 4126.821021

taxa_cupom = round((((1+cupom)**0.5)-1),8)

if vencimento.year%2 == 0:
    ano_vencimento ='par'
else:
    ano_vencimento ='impar'

if ano_vencimento == 'par':
    if hoje.month < 2:
        fluxo_1_par = '15/02/' + str(hoje.year)
    if hoje.month == 2 and hoje.day < 15:
        fluxo_1_par = '15/02/' + str(hoje.year)
    if hoje.month == 2 and hoje.day >= 15:
        fluxo_1_par = '15/08/' + str(hoje.year)
    if hoje.month > 2 and hoje.month < 8:
        fluxo_1_par = '15/08/' + str(hoje.year)
    if hoje.month == 8 and hoje.day < 15:
        fluxo_1_par = '15/08/' + str(hoje.year)
    if hoje.month == 8 and hoje.day >= 15:
        fluxo_1_par = '15/02/' + (str(hoje.year + 1))
    if hoje.month > 8:
        fluxo_1_par = '15/02/' + (str(hoje.year + 1))  
        
    fluxo_total = []
    fluxo_total.append(fluxo_1_par)
    fluxo_1_par = datetime.strptime(fluxo_1_par,'%d/%m/%Y').date()
        
    while(fluxo_1_par < vencimento):
        fluxo_n_par = '15/' + (str(fluxo_1_par.month + 6)) + '/' + str(fluxo_1_par.year)
        if float(str(fluxo_1_par.month + 6)) > 8:
            fluxo_n_par = '15/02/' + (str(fluxo_1_par.year + 1))
        else:
            fluxo_n_par = '15/08/' + str(fluxo_1_par.year)
            
        fluxo_total.append(fluxo_n_par)
        fluxo_1_par = datetime.strptime(fluxo_n_par,'%d/%m/%Y').date()

if ano_vencimento == 'impar':
    if hoje.month < 5:
        fluxo_2 = '15/05/' + str(hoje.year)
    if hoje.month == 5 and hoje.day < 15:
        fluxo_2 = '15/05/' + str(hoje.year)
    if hoje.month == 5 and hoje.day >= 15:
        fluxo_2 = '15/11/' + str(hoje.year)
    if hoje.month > 5 and hoje.month < 11:
        fluxo_2 = '15/11/' + str(hoje.year)
    if hoje.month == 11 and hoje.day < 15:
        fluxo_2 = '15/11/' + str(hoje.year)
    if hoje.month == 8 and hoje.day >= 15:
        fluxo_2 = '15/05/' + (str(hoje.year + 1))
    if hoje.month > 8:
        fluxo_2 = '15/05/' + (str(hoje.year + 1))
        
    fluxo_total = []
    fluxo_total.append(fluxo_2)
    fluxo_2 = datetime.strptime(fluxo_2,'%d/%m/%Y').date()
        
    while(fluxo_2 < vencimento):
        fluxo_n = '15/' + (str(fluxo_2.month + 6)) + '/' + str(fluxo_2.year)
        if float(str(fluxo_2.month + 6)) > 11:
            fluxo_n = '15/05/' + (str(fluxo_2.year + 1))
        else:
            fluxo_n = '15/11/' + str(fluxo_2.year)
            
        fluxo_total.append(fluxo_n)
        fluxo_2 = datetime.strptime(fluxo_n,'%d/%m/%Y').date()
            
fluxo_pgto = []
for data in fluxo_total:
    data = datetime.strptime(data,'%d/%m/%Y').date()
    data = wd.workdays(wd.workdays(data,-1),+1)
    fluxo_pgto.append(data)
                
diaano_pgto = []
for data in fluxo_pgto:
    data = truncate((wd.networkdays(hoje,data,252))/252,14)
    diaano_pgto.append(data)
                
pg_cupom = []
for data in diaano_pgto:
    valor_cupom = (taxa_cupom*100)/(1+i/100)**data
    pg_cupom.append(valor_cupom)
        
valor_principal = 100/(1+i/100)**(diaano_pgto[-1])
cotacao = truncate(pd.DataFrame(pg_cupom).sum() + valor_principal,4)/100
PU = truncate(cotacao*vna_ipca,6)
            
PU

4342.897242